In [1]:
# !pip install openai langchain tiktoken langchain_community
!pip install langchain tiktoken openai faiss-cpu pypdf chromadb langchain_community
!pip install wget PyMuPDF
!pip install gradio
!pip install nest_asyncio
!pip install requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 405.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 25.5 MB/s 

In [2]:
import openai

openai.__version__

'1.37.0'

In [16]:
import os

from google.colab import userdata
try:
  os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
  openai.api_key = os.getenv("OPENAI_API_KEY")
except:

  input_open_ai_key = input("OPENAI_API_KEY를 입력하세요")

  openai.OpenAI(api_key =input_open_ai_key )


In [17]:
import requests
import fitz  # PyMuPDF

# Jupyter Notebook에서 이벤트 루프 중첩 허용
nest_asyncio.apply()


# PDF 파일 다운로드 함수
def download_pdf(url, output_path):
    try:
        response = requests.get(url)
        response.raise_for_status()  # HTTP 오류가 발생했는지 확인
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {url} to {output_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")
        raise

# PDF 파일 내용 읽기 함수
def read_pdf(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_path} does not exist")
    try:
        document = fitz.open(file_path)
        text = ""
        for page_num in range(len(document)):
            page = document.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error reading {file_path}: {str(e)}")
        raise

In [18]:

import openai

# 답변 생성 함수
def analyze_saju_mbti(birth_date, birth_time, gender, name, mbti, documents):
    system_role = """
    너는 사용자가 제공한 생년월일과 MBTI 유형을 기반으로 사주와 MBTI 정보를 종합적으로 분석하는 도우미야.
    너의 역할은 사용자의 생년월일과 MBTI 유형을 바탕으로 성격과 성향, 그리고 직업이나 취향에 대해 자세히 설명하는 것이야.
    """

    documents_content = "\n".join([f"Document {i+1}:\n{doc}" for i, doc in enumerate(documents)])


    user_content = f'''Given the question: "내 생년월일은 {birth_date} {birth_time}이고, 성별은 {gender}입니다. 이름은 {name}이고, MBTI는 {mbti}입니다. 제 사주와 MBTI 정보를 바탕으로 내가 어떤 사람인지, 어떤 운세를 가지고 있는지, 어떤 성격을 가지고 있는지,
    어떤 능력을 가지고 있는지, 어떤 궁합을 가지고 있는지, 사주와 MBTI 를 복합적으로 분석해 주세요." Please analyze the given birth date, birth time, gender, name, and MBTI to predict the user's MBTI personality type based on traditional Saju principles combined with MBTI statistical correlations. The response should be in Korean.

1. **Birth Date:** {birth_date}
2. **Birth Time:** {birth_time}
3. **Gender:** {gender}
4. **Name:** {name}
5. **MBTI:** {mbti}


    """
'''

    response = openai.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {'role': 'system', 'content': system_role},
            {'role': 'user', 'content': user_content}
        ]
    )
    # print(response.choices[0].message.content)
    return response.choices[0].message.content
    # return response.choices[0].message['content']

In [19]:
import nest_asyncio
import asyncio

# Jupyter Notebook에서 이벤트 루프 중첩 허용
nest_asyncio.apply()


# 테스트 함수
async def test_analyze_saju_mbti():
    # PDF 파일 목록 및 경로
    pdf_urls = [
        # "https://raw.githubusercontent.com/pplkjh/saju_mbti_bot/main/%EC%82%AC%EC%A3%BC%20%EB%AA%85%EB%A6%AC%20%EC%9D%BC%EC%A3%BC%EB%A1%A0%EA%B3%BC%20MBTI%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95%EC%9D%98%20%EC%83%81%ED%98%B8%EB%B3%B4%EC%99%84%EC%84%B1%EC%97%90%20%EA%B4%80%ED%95%9C%20%EC%97%B0%EA%B5%AC.pdf",
        # "https://raw.githubusercontent.com/pplkjh/saju_mbti_bot/main/%EC%82%AC%EC%A3%BC%20%EB%AA%85%EB%A6%AC%ED%95%99%EC%9D%98%208%EA%B0%80%EC%A7%80%20%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95%EA%B3%BC%20MBTI%EA%B8%B0%EB%8A%A5%EB%B3%84%208%EA%B0%80%EC%A7%80%20%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95%EC%9D%98%20%EC%83%81%EA%B4%80%EC%97%B0%EA%B5%AC.pdf",
        # "https://raw.githubusercontent.com/pplkjh/saju_mbti_bot/main/%EC%82%AC%EC%A3%BC%EB%AA%85%EB%A6%AC%ED%95%99%EC%9D%84%20%ED%86%B5%ED%95%9C%20%EC%84%B1%EA%B2%A9.pdf",
        # "https://raw.githubusercontent.com/pplkjh/saju_mbti_bot/main/%EC%82%AC%EC%A3%BC%EB%AA%85%EB%A6%AC%ED%95%99%EC%9D%98%208%EA%B0%80%EC%A7%80%20%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95%EA%B3%BC%20MBTI%20%EA%B8%B0%EB%8A%A5%EB%B3%84%208%EA%B0%80%EC%A7%80%20%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95%EC%9D%98%20%EC%83%81%EA%B4%80%EC%97%B0%EA%B5%AC.pdf",
        # "https://raw.githubusercontent.com/pplkjh/saju_mbti_bot/main/%EC%82%AC%EC%A3%BC%EC%99%80%20MBTI%20%EC%84%B1%EA%B2%A9%EC%9D%B4%EB%A1%A0%EA%B3%BC%EC%9D%98%20%EC%83%81%EA%B4%80%EA%B4%80%EA%B3%84%20%EC%97%B0%EA%B5%AC.pdf",
        # "https://raw.githubusercontent.com/pplkjh/saju_mbti_bot/main/%EC%97%AD%EA%B3%BC%2B%EB%AA%85%EB%A6%AC%ED%95%99.pdf"
    ]

    # pdf_files = ["document1.pdf", "document2.pdf","document3.pdf","document4.pdf","document5.pdf" ]
    pdf_files = []  # """ 현제 토큰 문제로 pdf 추가 불가, 추후 langchain 으로 vectorstore 에 벡터화해서 저장해서 pdf 내용 기능 추가 """

    # 문서 내용을 저장할 리스트
    documents = []

    # PDF 파일을 다운로드하고 내용을 읽어옴
    for url, file_path in zip(pdf_urls, pdf_files):
        download_pdf(url, file_path)
        documents.append(read_pdf(file_path))

    # 임의의 쿼리로 테스트
    birth_date = "1990-05-21"
    birth_time = "14:30"
    gender = "여성"
    name = "홍길동"
    mbti = "INFJ"

    result =  return_answer(birth_date, birth_time, gender, name, mbti, documents)
    print(result)

# 테스트 실행
asyncio.run(test_analyze_saju_mbti()) # Use asyncio.run to execute the coroutine

# test_analyze_saju_mbti()

홍길동님, 주어진 정보에 기초하여 여러 가지 측면으로 분석해보겠습니다.

### **사주 분석:**
**1. 운세:**
- 홍길동님의 사주를 살펴보면, 태양자리에 천황성(金)이 위치하여 권위와 성공을 상징합니다. 또한, 천마성(土)이 겉으로 밝게 비치어 예지력과 결단력을 가진 분이시겠군요. 큰 목표를 이루기 위해 달려가는 성향을 가지고 계실 것으로 보입니다.

**2. 성격:**
- 사주에서 보았을 때, 천황자리에 놓치(하느달)가 있는데, 이는 홍길동님이 배려심 깊고 정서적이며 온화한 성품을 갖고 있다는 것을 의미합니다. 냉철한 판단력을 통해 주변을 잘 살피고 배려하는 모습이 돋보일 것으로 예상됩니다.

**3. 능력:**
- 잘 발달된 천마성(土)은 홍길동님이 조직적이고 체계적으로 일을 처리하며 규칙적이고 주도적으로 문제를 해결할 수 있는 재능을 가지고 있다는 것을 시사합니다. 분야를 가리지 않고 다방면에 능력을 발휘할 수 있을 것입니다.

### **MBTI(인프제) 성향 분석:**
- INFJ 성격 유형은 대담하고 통찰력이 뛰어나며 동정심이 강하고 이타적인 특징을 지닙니다. 주변 사람들에게 조언을 주는 데 능숙하며, 자기 성장과 타인의 발전을 추구합니다. 깊이 있는 사고와 예측 능력을 지녔으며, 고요한 운명론자로서 인생을 심도있게 살펴보는 경향이 있습니다.

### **사주와 MBTI 복합 분석:**
- 홍길동님의 사주와 MBTI 결과를 종합해보면, 권위와 예지력, 그리고 배려심과 동정심을 갖춘 다재다능한 분이시며, 능동적인 리더십을 발휘하면서도 주변을 존중하고 따뜻한 마음을 가지고 있을 것으로 판단됩니다. 자신의 가치관을 중시하면서도 타인을 이해하는 관계형 성향이 돋보이며, 과거를 공부하고 미래를 준비하는 등 체계적으로 성장하는 모습을 보일 것으로 예측됩니다.

결과적으로, 홍길동님은 권위와 예지력을 결합한 배려심과 세심한 마음씨를 가진 다재다능한 리더로써, 주변 사람들과의 긍정적인 인연을 통해 더 높은 성장을 이룰 수 있을 것으로 기대됩니다.


In [28]:

# 그라디오 인터페이스 함수
async def analyze_saju_mbti(birth_date, birth_time, gender, name, mbti):
    # PDF 파일 목록 및 경로
    pdf_urls = [
        # "https://raw.githubusercontent.com/pplkjh/saju_mbti_bot/main/%EC%82%AC%EC%A3%BC%20%EB%AA%85%EB%A6%AC%20%EC%9D%BC%EC%A3%BC%EB%A1%A0%EA%B3%BC%20MBTI%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95%EC%9D%98%20%EC%83%81%ED%98%B8%EB%B3%B4%EC%99%84%EC%84%B1%EC%97%90%20%EA%B4%80%ED%95%9C%20%EC%97%B0%EA%B5%AC.pdf",
        # "https://raw.githubusercontent.com/pplkjh/saju_mbti_bot/main/%EC%82%AC%EC%A3%BC%20%EB%AA%85%EB%A6%AC%ED%95%99%EC%9D%98%208%EA%B0%80%EC%A7%80%20%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95%EA%B3%BC%20MBTI%EA%B8%B0%EB%8A%A5%EB%B3%84%208%EA%B0%80%EC%A7%80%20%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95%EC%9D%98%20%EC%83%81%EA%B4%80%EC%97%B0%EA%B5%AC.pdf",
        # "https://raw.githubusercontent.com/pplkjh/saju_mbti_bot/main/%EC%82%AC%EC%A3%BC%EB%AA%85%EB%A6%AC%ED%95%99%EC%9D%84%20%ED%86%B5%ED%95%9C%20%EC%84%B1%EA%B2%A9.pdf",
        # "https://raw.githubusercontent.com/pplkjh/saju_mbti_bot/main/%EC%82%AC%EC%A3%BC%EB%AA%85%EB%A6%AC%ED%95%99%EC%9D%98%208%EA%B0%80%EC%A7%80%20%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95%EA%B3%BC%20MBTI%20%EA%B8%B0%EB%8A%A5%EB%B3%84%208%EA%B0%80%EC%A7%80%20%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95%EC%9D%98%20%EC%83%81%EA%B4%80%EC%97%B0%EA%B5%AC.pdf",
        # "https://raw.githubusercontent.com/pplkjh/saju_mbti_bot/main/%EC%82%AC%EC%A3%BC%EC%99%80%20MBTI%20%EC%84%B1%EA%B2%A9%EC%9D%B4%EB%A1%A0%EA%B3%BC%EC%9D%98%20%EC%83%81%EA%B4%80%EA%B4%80%EA%B3%84%20%EC%97%B0%EA%B5%AC.pdf",
        # "https://raw.githubusercontent.com/pplkjh/saju_mbti_bot/main/%EC%97%AD%EA%B3%BC%2B%EB%AA%85%EB%A6%AC%ED%95%99.pdf"
    ]

    # pdf_files = ["document1.pdf", "document2.pdf","document3.pdf","document4.pdf","document5.pdf" ]
    pdf_files = []  # """ 현제 토큰 문제로 pdf 추가 불가, 추후 langchain 으로 vectorstore 에 벡터화해서 저장해서 pdf 내용 기능 추가 """

    # 문서 내용을 저장할 리스트
    documents = []

    # PDF 파일을 다운로드하고 내용을 읽어옴
    for url, file_path in zip(pdf_urls, pdf_files):
        download_pdf(url, file_path)
        documents.append(read_pdf(file_path))

    # 임의의 쿼리로 테스트
    birth_date = "1990-05-21"
    birth_time = "14:30"
    gender = "여성"
    name = "홍길동"
    mbti = "INFJ"

    result =  await return_answer(birth_date, birth_time, gender, name, mbti, documents)
    return result


import gradio as gr
interface = gr.Interface(
    fn=analyze_saju_mbti,
    inputs=[
        gr.components.Textbox(label="생년월일 (예: 1990-05-21)"), # Use gr.components.Textbox instead of gr.inputs.Textbox
        gr.components.Textbox(label="출생 시간 (예: 14:30)"),
        gr.components.Radio(label="성별", choices=["남성", "여성"]),
        gr.components.Textbox(label="이름"),
        gr.components.Textbox(label="MBTI (예: INFJ)")
    ],
    outputs="text",
    title="사주와 MBTI 분석",
    description="생년월일, 출생 시간, 성별, 이름, 그리고 MBTI를 입력하여 사주와 MBTI를 분석합니다."
)

# Gradio 인터페이스 실행
interface.launch()

Exception in thread Thread-102 (run):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 40, in _get_event_loop
    loop = events.get_event_loop_policy().get_event_loop()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 67, in get_event_loop
    _patch_loop(loop)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 193, in _patch_loop
    raise ValueError('Can\'t patch loop of type %s' % type(loop))
ValueError: Can't patch loop of type <class 'uvloop.L

KeyboardInterrupt: 

In [ ]:

# 그라디오 인터페이스 함수
async def analyze_saju_mbti(birth_date, birth_time, gender, name, mbti):
    # PDF 파일 목록 및 경로
    pdf_urls = [
        "https://raw.githubusercontent.com/pplkjh/saju_mbti_bot/main/%EC%82%AC%EC%A3%BC%20%EB%AA%85%EB%A6%AC%20%EC%9D%BC%EC%A3%BC%EB%A1%A0%EA%B3%BC%20MBTI%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95%EC%9D%98%20%EC%83%81%ED%98%B8%EB%B3%B4%EC%99%84%EC%84%B1%EC%97%90%20%EA%B4%80%ED%95%9C%20%EC%97%B0%EA%B5%AC.pdf"
    ]

    pdf_files = ["document1.pdf"]

    # 문서 내용을 저장할 리스트
    documents = []

    # PDF 파일을 다운로드하고 내용을 읽어옴
    for url, file_path in zip(pdf_urls, pdf_files):
        download_pdf(url, file_path)
        documents.append(read_pdf(file_path))

    # 문서 내용을 분할하여 저장
    chunks = []
    for document in documents:
        chunks.extend(split_document(document))

    # 임베딩 생성
    docsearch = FAISS.from_texts(chunks, OpenAIEmbeddings())

    result = await return_answer(birth_date, birth_time, gender, name, mbti, docsearch)
    return result

# 그라디오 인터페이스 설정
iface = gr.Interface(
    fn=analyze_saju_mbti,
    inputs=[
        gr.Textbox(label="생년월일 (예: 1990-05-21)"),
        gr.Textbox(label="출생 시간 (예: 14:30)"),
        gr.Radio(label="성별", choices=["남성", "여성"]),
        gr.Textbox(label="이름"),
        gr.Textbox(label="MBTI (예: INFJ)")
    ],
    outputs="text",
    title="사주와 MBTI 분석",
    description="생년월일, 출생 시간, 성별, 이름, MBTI를 입력하면 사주와 MBTI 정보를 바탕으로 분석 결과를 제공합니다."
)

# 인터페이스 실행
iface.launch()

In [25]:
import gradio as gr

# 그라디오 인터페이스 설정
iface = gr.Interface(
    fn=analyze_saju_mbti,
    inputs=[
        gr.Textbox(label="생년월일 (예: 1990-05-21)"),
        gr.Textbox(label="출생 시간 (예: 14:30)"),
        gr.Radio(label="성별", choices=["남성", "여성"]),
        gr.Textbox(label="이름"),
        gr.Textbox(label="MBTI (예: INFJ)")
    ],
    outputs="text",
    title="사주와 MBTI 분석",
    description="생년월일, 출생 시간, 성별, 이름, MBTI를 입력하면 사주와 MBTI 정보를 바탕으로 분석 결과를 제공합니다."
)

# 인터페이스 실행
iface.launch()

Exception in thread Thread-92 (run):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 40, in _get_event_loop
    loop = events.get_event_loop_policy().get_event_loop()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 67, in get_event_loop
    _patch_loop(loop)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 193, in _patch_loop
    raise ValueError('Can\'t patch loop of type %s' % type(loop))
ValueError: Can't patch loop of type <class 'uvloop.Lo

KeyboardInterrupt: 

In [ ]:
    user_content = f'''Given the question: "내 생년월일은 {birth_date} {birth_time}이고, 성별은 {gender}입니다. 이름은 {name}이고, MBTI는 {mbti}입니다. 제 사주와 MBTI 정보를 바탕으로 내가 어떤 사람인지, 어떤 운세를 가지고 있는지, 어떤 성격을 가지고 있는지,
    어떤 능력을 가지고 있는지, 어떤 궁합을 가지고 있는지, 사주와 MBTI 를 복합적으로 분석해 주세요." Please analyze the given birth date, birth time, gender, name, and MBTI to predict the user's MBTI personality type based on traditional Saju principles combined with MBTI statistical correlations. The response should be in Korean.

1. **Birth Date:** {birth_date}
2. **Birth Time:** {birth_time}
3. **Gender:** {gender}
4. **Name:** {name}
5. **MBTI:** {mbti}

### Considerations for the Response:
1. **Explain the Process:** Briefly describe how Saju principles (which are based on the Chinese zodiac, elements, and other astrological factors) and the user's name are combined with MBTI statistical correlations to make the prediction.
2. **Prediction:** Provide the most likely MBTI type(s) for the user based on the analysis.
3. **Accuracy:** Mention that the prediction is based on general correlations and should be taken as a fun and interesting insight rather than a definitive result.

### Documents:
{documents_content}

### Example Response (in Korean)
    {gender}인 {name} 님의 생년월일시는 {birth_date}, {birth_time}이고 MBTI가 {mbti}라면, 이 성격 유형에 맞는 사주 요소를 고려하여 사주와 성격을 분석해보겠습니다.

    {mbti}의 주요 특성:
    - E (외향적): 사람들과의 상호작용을 즐기며, 에너지를 외부에서 얻습니다.
    - N (직관적): 전체적인 그림과 가능성을 보고, 창의적이고 미래 지향적입니다.
    - T (사고적): 논리적이고 분석적으로 생각하며, 객관적인 결정을 내립니다.
    - P (인식적): 유연하고 자율적인 접근을 선호하며, 즉흥적이고 개방적입니다.

    사주 요소:
    {mbti}의 특성을 반영한 사주 요소를 고려해보면, 에너지 넘치고 창의적이며 논리적이고 유연한 성향이 강한 날에 태어났을 가능성이 큽니다. 이를 반영하여 생년월일을 추정해보겠습니다.

    추정 생년월일:
    - 태어난 해: 에너지가 넘치고 활기찬 성향을 가진 해를 선택할 것입니다.
      예: 병인년 (丙寅, 불호랑이해), 갑진년 (甲辰, 나무용해)
    - 태어난 달: 창의적이고 변화를 즐기는 성향을 가진 달을 선택할 것입니다.
      예: 인월 (寅月, 호랑이달, 2월), 진월 (辰月, 용달, 4월)
    - 태어난 일: 독립적이고 논리적인 성향을 가진 날을 선택할 것입니다.
      예: 경진일 (庚辰, 금용일), 갑인일 (甲寅, 나무호랑이일)
    - 태어난 시간: 유연하고 즉흥적인 성향을 가진 시간을 선택할 것입니다.
      예: 오시 (午時, 말시간, 11시-13시), 신시 (申時, 원숭이시간, 15시-17시)

    예시 생년월일:
    이 요소들을 종합하여 예시 생년월일을 추정해보면 다음과 같이 설정할 수 있습니다:

    생년월일: 1990년 4월 10일 오후 3시
    - 태어난 해: 갑진년 (甲辰, 나무용해)
    - 태어난 달: 진월 (辰月, 용달, 4월)
    - 태어난 일: 갑인일 (甲寅, 나무호랑이일)
    - 태어난 시간: 신시 (申時, 원숭이시간, 15시)

    이 생년월일은 {mbti}의 성향과 잘 어울리는 에너지 넘치고 창의적이며 논리적이고 유연한 성격을 반영합니다. 실제 생년월일과는 차이가 있을 수 있지만, {mbti}의 주요 특성을 바탕으로 사주 요소를 고려한 추정치입니다.
    """
'''

In [16]:
def return_answer(query, input_text=''):
    system_role = """
    당신은 사주와 MBTI를 분석하여 사용자의 성격 유형을 예측하는 AI 봇입니다. 사용자가 생년월일, 성별, 이름을 입력하면 이를 바탕으로 사주와 MBTI의 통계적 상관관계를 분석하여 MBTI 유형을 예측합니다.
    답변은 한국어로 제공하며, 정중하고 공식적인 문체를 사용합니다.
    주어진 문서를 참고하여 질문에 답변하고, 문서에서 관련 정보를 찾을 수 없으면 문서를 참고하지 않고 답변을 작성하세요.
    """

    user_content = f'''Given the question: "{str(query)}". Please analyze the given birth date, gender, and name to predict the user's MBTI personality type based on traditional Saju principles combined with MBTI statistical correlations. The response should be in Korean.

1. **Birth Date:** {query.get('birth_date')}
2. **Gender:** {query.get('gender')}
3. **Name:** {query.get('name')}

### Considerations for the Response:
1. **Explain the Process:** Briefly describe how Saju principles (which are based on the Chinese zodiac, elements, and other astrological factors) and the user's name are combined with MBTI statistical correlations to make the prediction.
2. **Prediction:** Provide the most likely MBTI type(s) for the user based on the analysis.
3. **Accuracy:** Mention that the prediction is based on general correlations and should be taken as a fun and interesting insight rather than a definitive result.

### Example Response (in Korean)
사용자가 제공한 생년월일, 성별, 그리고 이름을 바탕으로 MBTI 유형을 예측한 결과는 다음과 같습니다:

"사주(사주팔자)에 따르면, 당신의 생년월일인 {query.get('birth_date')}과 이름 {query.get('name')}은 특정한 요소와 동물 띠에 해당하여 성격에 대한 통찰을 제공합니다. 이러한 통찰을 MBTI 연구에서 발견된 통계적 상관관계와 결합하여 예측한 결과, 당신의 MBTI 유형은 **INFJ**일 가능성이 높습니다. INFJ 유형은 공감능력, 직관력, 그리고 타인을 깊이 이해하는 능력이 뛰어납니다. 하지만 이 예측은 일반적인 패턴을 기반으로 한 것이므로 재미로 참고하시길 바랍니다. 보다 정확한 결과를 원하신다면, 표준화된 MBTI 검사를 받는 것을 추천합니다."

이 과정을 즐기시고, 이를 통해 자신의 성격을 탐구하는 재미를 느껴보세요!
'''

    response = openai.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {'role': 'system', 'content': system_role},  # 역할부여
            {'role': 'user', 'content': user_content}
        ]
    )
    return response.choices[0].message.content